In [ ]:
f1 = [1,14,13,12,7,6,15,5,11,9,10,16,3,2,4,8]
f2 = [1,14,13,12,15,5,7,6,11,9,10,16,3,2,4,8]
def DiscreteDistance(arr1:list[int], arr2:list[int]):
    count =0
    n = len(arr1)
    for i in np.arange(n-1):
        if arr1[i] != arr2[i] and arr1[i+1] != arr2[i+1]:
            count +=1
    return (count/n)*10

In [ ]:
DiscreteDistance(f1,f2)